<a href="https://colab.research.google.com/github/guiraposo/GRpy/blob/main/TOV_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install sympy
!git clone https://github.com/guiraposo/GRpy

fatal: destination path 'GRpy' already exists and is not an empty directory.


In [21]:
import sys
sys.path.insert(0,'/content/GRpy')

In [19]:
import sympy as sp
import GR_Tensors  as grt

In [4]:
def create_symbols():
    # Definition of spacetime coordinates
    t, theta, phi = sp.symbols('t theta phi', reals=True) #time, and angular coodinates
    r = sp.symbols('r', positive=True) # radial coordinate is positive always

    # Definition of other symbolic parameters that are relevant
    M = sp.symbols('M', positive=True) # Mass of the star or compact object
    radius = sp.symbols('radius', positive=True) # Radius of the star or compact object.
    return t, theta, phi, r, M, radius

def define_functions(r):
    alpha = sp.Function('alpha')(r) # relativistic gravitational potential
    m = sp.Function('m')(r)         # mass function
    rho = sp.Function('rho')(r)     # Energy density
    p = sp.Function('p')(r)         # Pressure
    return alpha, m, rho, p

def metric_tensor(r, theta, alpha, m):
    g = sp.Matrix([[- sp.exp(2*alpha), 0, 0, 0],
                   [0, (1 - 2*m/r)**(-1), 0, 0],
                   [0, 0, r**2, 0],
                   [0, 0, 0, r**2 * sp.sin(theta)**2]]) #Spherically symmetric line element
    return g

def four_velocity(g):
    u_t = -sp.sqrt(-g[0, 0])  # Time component of the four-velocity, assuming normalization
    u = [u_t, 0, 0, 0]  # Four-velocity components in static spacetime
    return u

# Main script
t, theta, phi, r, M, radius = create_symbols() # Create coordinates and other important symbols
alpha, m, rho, p = define_functions(r) # Create function for the metric and for the matter
g = metric_tensor(r, theta, alpha, m)  # Covariant metric tensor
u = four_velocity(g)                   # Contravariant 4 - velocity


In [5]:
t, theta, phi, r, M, radius = create_symbols() # Create coordinates and other important symbols
alpha, m, rho, p = define_functions(r) # Create function for the metric and for the matter
g = metric_tensor(r, theta, alpha, m)  # Covariant metric tensor
u = four_velocity(g)                   # Contravariant 4 - velocity


In [6]:
coords = [t, r, theta, phi]  # Assuming these are your coordinates
christoffel_symbols_g = grt.christoffel_symbols(g, coords)
riemann_tensor_g = grt.riemann_tensor(christoffel_symbols_g, coords)
ricci_tensor_g = grt.ricci_tensor(riemann_tensor_g, coords)
ricci_scalar_g = grt.ricci_scalar(ricci_tensor_g, g)
einstein_tensor_g = grt.einstein_tensor(ricci_tensor_g, ricci_scalar_g, g)
Tdd_g = grt.stress_energy_tensor_perfect_fluid(rho, p, u, g)
einstein_eqs_g = grt.einsteins_equations(einstein_tensor_g, Tdd_g, g)

In [7]:
alpha_test = 1/2 * sp.log(1 - 2*M / r)
m_test = M
rho_test = 0
p_test = 0
christoffel_symbols_substituted = [[[
    sp.simplify(symbol.subs({alpha: alpha_test, m: m_test}))
    for symbol in row]
    for row in matrix]
    for matrix in christoffel_symbols_g]


In [27]:
sol1 = sp.solve([einstein_eqs_g[1,1], einstein_eqs_g[0,0]] ,(sp.diff(alpha, r),sp.diff(m, r)))
print(sol1)

{Derivative(alpha(r), r): 4*pi*r**3*p(r)/(r**2 - 2*r*m(r)) + m(r)/(r**2 - 2*r*m(r)), Derivative(m(r), r): 4*pi*r**2*rho(r)}


In [30]:
sol1[sp.diff(alpha, r)]

4*pi*r**3*p(r)/(r**2 - 2*r*m(r)) + m(r)/(r**2 - 2*r*m(r))

In [ ]:
sp.simplify(sp.simplify(ricci_scalar_g.subs({alpha: alpha_test, m: m_test})))

0

In [ ]:
sp.simplify(sp.simplify(einstein_tensor_g.subs({alpha: alpha_test, m: m_test})))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [ ]:
Tdd_g

Matrix([
[rho*A(r)**2,         0,      0,                    0],
[          0, p*B(r)**2,      0,                    0],
[          0,         0, p*r**2,                    0],
[          0,         0,      0, p*r**2*sin(theta)**2]])

In [ ]:
einstein_eqs_g

Matrix([
[(-8*pi*r**2*rho(r) + 2.0*Derivative(m(r), r))*exp(2*alpha(r))/r**2,                                                                                                                                0,                                                                                                                                                                                                                                                                                                                                             0,                                                                                                                                                                                                                                                                                                                                                                                                                    0],
[                                                 

In [ ]:
sp.simplify(sp.simplify(einstein_eqs_g.subs({A: A_test, B: B_test, rho: rho_test, p: p_test})))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])